Script to get the tissue region percentage for images

Importing packages

In [ ]:
import numpy as np
import pandas as pd
import tifffile
import cv2
from tqdm import tqdm
import os
import matplotlib.pyplot as plt


In [ ]:
data = pd.read_csv("data.csv")

In [ ]:
data['tissue_name'].value_counts()

In [ ]:
len(data)

In [ ]:
local_directory_path = "path/to/data"

In [ ]:
data.columns

In [ ]:
data_recording = []
for row in tqdm(data.iterrows(),total=len(data)):
    image_path =local_directory_path +"/".join(row[1]['image'].split("/")[-2:])
    tissue_region_percentage = 0
    if os.path.isfile(image_path):
        image = tifffile.imread(image_path)
        grayscale_image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
        _, thresholded_image = cv2.threshold(grayscale_image,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        tissue_region_percentage = np.sum((thresholded_image==255)*1.0)/(thresholded_image.shape[0]*thresholded_image.shape[1])*100
        data_recording.append([row[1]['tissue_name'],row[1]['age'],row[1]['sex'],row[1]['image'],image_path,tissue_region_percentage,True])
    else:
        data_recording.append([row[1]['tissue_name'],row[1]['age'],row[1]['sex'],row[1]['image'],image_path,tissue_region_percentage,False])

df = pd.DataFrame(data_recording,columns=list(data.columns) + ['local_path','tissue_region_percentage',"Image_exist?"])
df.to_csv('data_with_tissue_region_percentage.csv',index=None)